# ST2195 Coursework
## Loading libraries and data
#### Load all libraries needed

In [ ]:
import glob
import datetime
import calendar
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import calplot
import networkx as nx
import seaborn as sns
from sklearn import *
import statsmodels.api as sm
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV      
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier 

# Data Wrangling

Download airports.csv, carriers.csv, plane-data.csv, 2006.csv.bz2, and 2007.csv.bz2 from the Harvard dataverse at <https://doi.org/10.7910/DVN/HG7NV7>. Save the files in a folder called <code> dataverse_files </code>. Create data frames called <code>airports</code> with airports.csv, <code>carriers</code> with carriers.csv, and <code>planes</code> with plane-data.csv.

In [ ]:
airports = pd.read_csv("./dataverse_files/airports.csv")
carriers = pd.read_csv("./dataverse_files/carriers.csv")
planes = pd.read_csv("./dataverse_files/plane-data.csv")

Create data frame called <code>ontime</code> with 2006.csv.bz2, 2007.csv.bz2.

In [ ]:
path = "./dataverse_files"
files = glob.glob(path + "/*.csv.bz2")

l = []

for f in files:
    tempontime = pd.read_csv(f)
    l.append(tempontime)
    print(f'Data frame for {f}, is successfully created with shape {tempontime.shape}')
    ontime = pd.concat(l, axis=0)
    
    print(ontime.shape)
    ontime.head()

In the <code>ontime</code> data frame, convert all the column names into lower case. Remove duplicated rows. 
Rename <code> dayofmonth </code> column as <code>day</code>

In [ ]:
ontime = ontime.rename(columns = str.lower)
ontime.drop_duplicates(keep = "first", inplace = True)
ontime.rename(columns = {"dayofmonth":"day"}, inplace = True)

Convert the values in <code> month </code> and <code> day </code> as 2 digits.  Create a new column called <code> date </code> in year-month-day format.Concatenate the values of year,month and day.

In [ ]:
for column in ontime[["month", "day"]]:   
    ontime[column] = ontime[column].apply(lambda x: str(x).zfill(2))

ontime["date"] = ontime["year"].astype(str) + ontime["month"]+ ontime["day"]
ontime["date"] = pd.to_datetime(ontime["date"], format= "%Y-%m-%d")

Drop non-applicable values in <code> deptime, arrtime, crsdeptime and crsarrtime </code>

In [ ]:
ontime.dropna(subset=["deptime", "arrtime","crsdeptime","crsarrtime"], inplace = True)

Convert the timing of departure, arrival, scheduled departure and scheduled arrival timing from 2400 to 0.

In [ ]:
for column in ontime[["deptime", "arrtime","crsdeptime","crsarrtime"]]:   
    ontime[column] = ontime[column].apply(lambda x: str(x))

for column in ontime[["deptime", "arrtime","crsdeptime","crsarrtime"]]:
    ontime.loc[ontime[column] == "2400", column] = "0"
    
ontime = ontime.astype({"deptime":"float","arrtime":"float", "crsdeptime":"float","crsarrtime":"float"})
ontime = ontime.astype({"deptime":"int","arrtime":"int", "crsdeptime":"int","crsarrtime":"int"})

Now, exclude all the departure, arrival, scheduled departure and scheduled arrival timing more than 2400.

In [ ]:
for column in ontime[["deptime", "arrtime","crsdeptime","crsarrtime"]]:
    ontime = ontime[ontime[column] < 2400] 

Divide time into 4-hours intervals and name the intervals as [00:00-04:00), [04:00-08:00), [08:00-12:00) ,
[12:00-16:00), [16:00-20:00) , [20:00-24:00).

In [ ]:
ontime["deptimeintervals"] = pd.cut(x = ontime["deptime"], right = False, 
                                    bins = [0,400,800,1200,1600,2000,2400],
                                    labels = ["[00:00-04:00)","[04:00-08:00)","[08:00-12:00)",
                                              "[12:00-16:00)","[16:00-20:00)", "[20:00-24:00)"])

ontime["arrtimeintervals"] = pd.cut(x = ontime["arrtime"], right = False, 
                                    bins = [0,400,800,1200,1600,2000,2400],
                                    labels = ["[00:00-04:00)","[04:00-08:00)","[08:00-12:00)",
                                              "[12:00-16:00)","[16:00-20:00)", "[20:00-24:00)"])
        
ontime["crsdeptimeintervals"] = pd.cut(x = ontime["crsdeptime"], right = False, 
                                       bins = [0,400,800,1200,1600,2000,2400],
                                       labels = ["[00:00-04:00)","[04:00-08:00)","[08:00-12:00)",
                                                 "[12:00-16:00)","[16:00-20:00)", "[20:00-24:00)"])

ontime["crsarrtimeintervals"] = pd.cut(x = ontime["crsarrtime"], right = False, 
                                       bins = [0,400,800,1200,1600,2000,2400],
                                       labels = ["[00:00-04:00)","[04:00-08:00)","[08:00-12:00)",
                                                 "[12:00-16:00)","[16:00-20:00)", "[20:00-24:00)"])

Make sure there are 4 digits in <code> deptime </code> , <code> arrtime </code>, <code> crsdeptime </code> and <code> crsarrtime </code>

In [ ]:
for column in ontime[["deptime", "arrtime","crsdeptime","crsarrtime"]]:   
    ontime[column] = ontime[column].apply(lambda x: str(x).zfill(4))

Convert <code>deptime, arrtime, crsdeptime and crsarrtime </code> into hours and minutes.

In [ ]:
ontime["dephour"] = ontime.deptime.str[:2]
ontime["depmin"] = ontime.deptime.str[2:]
ontime["crsdephour"] = ontime.crsdeptime.str[:2]
ontime["crsdepmin"] = ontime.crsdeptime.str[2:]
ontime["arrhour"] = ontime.arrtime.str[:2]
ontime["arrmin"] = ontime.arrtime.str[2:]
ontime["crsarrhour"] = ontime.crsarrtime.str[:2]
ontime["crsarrmin"] = ontime.crsarrtime.str[2:]

Convert <code>deptime, arrtime, crsdeptime and crsarrtime </code> into hours:minutes format.

In [ ]:
ontime["deptime"] = pd.to_datetime(ontime["dephour"] + ":" 
                                   + ontime["depmin"], format="%H:%M").dt.time
ontime["arrtime"] = pd.to_datetime(ontime["arrhour"] + ":" 
                                   + ontime["arrmin"], format="%H:%M").dt.time
ontime["crsdeptime"] = pd.to_datetime(ontime["crsdephour"] + ":" 
                                   + ontime["crsdepmin"], format="%H:%M").dt.time
ontime["crsarrtime"] = pd.to_datetime(ontime["crsarrhour"] + ":" 
                                   + ontime["crsarrmin"], format="%H:%M").dt.time

Convert <code>deptime, arrtime, crsdeptime and crsarrtime</code> to timedelta64[ns] data type

In [ ]:
ontime["deptime2"] = pd.to_timedelta(pd.to_datetime(ontime["deptime"],format='%H:%M:%S').
                                     dt.strftime('%H:%M:%S'))
ontime["arrtime2"] = pd.to_timedelta(pd.to_datetime(ontime["arrtime"],format='%H:%M:%S').
                                     dt.strftime('%H:%M:%S'))
ontime["crsdeptime2"] = pd.to_timedelta(pd.to_datetime(ontime["crsdeptime"],format='%H:%M:%S').
                                        dt.strftime('%H:%M:%S'))
ontime["crsarrtime2"] = pd.to_timedelta(pd.to_datetime(ontime["crsarrtime"],format='%H:%M:%S').
                                        dt.strftime('%H:%M:%S'))

Concatenate <code> date </code> with <code> deptime, arrtime, crsdeptime, and crsarrtime </code> respectively.

In [ ]:
ontime["datedeptime"] = ontime["year"].astype(str) + (ontime["month"] + ontime["day"]+ ontime["dephour"] 
                                                      + ontime["depmin"])
ontime["datearrtime"] = ontime["year"].astype(str) + (ontime["month"] + ontime["day"]+ ontime["arrhour"] 
                                                      + ontime["arrmin"])
ontime["datecrsdeptime"] = ontime["year"].astype(str) + (ontime["month"] + ontime["day"]+ ontime["crsdephour"] 
                                                         + ontime["crsdepmin"])
ontime["datecrsarrtime"] = ontime["year"].astype(str) + (ontime["month"] + ontime["day"]+ ontime["crsarrhour"]
                                                         + ontime["crsarrmin"])

Convert <code> datedeptime, datearrtime, datecrsdeptime and datecrsarrtime </code> into datetime objects.

In [ ]:
for column in ontime[["datedeptime", "datearrtime","datecrsdeptime","datecrsarrtime"]]:
    ontime[column] = pd.to_datetime(ontime[column], format="%Y%m%d%H%M")

Create <code>biarrdelay</code> columns with value of 1 indicating delay and 0 depicting no delay.

In [ ]:
# A list of conditions
condition = [(ontime["arrdelay"] <= 0),(ontime["arrdelay"] > 0)]

# A list of valus to assign for each condition
values = [0, 1]

# Assign values in the list based on conditions using np.select
ontime["biarrdelay"] = np.select(condition, values)

In [ ]:
ontime

# 1.When is the best time of day, day of week and time of the year to fly to minimize delay?

### Idea: Only arrival delays are considered as departure delay did not necessarily result in arrival delay. Average arrival delays are computed for each date, month, day of week and each time interval to identify the period with shortest average arrival delay to fly to minimize delays.

### a. Calendar plots for average daily arrival delay for 2006 and 2007

For computation of arrival delay, drop non-applicable arrival delay values in ontime.
Filter out diverted and canceled flights. When the flight arrives early, it has a negative value. 
Replace all negative values of arrdelay columns with 0 to indicate no delay.

In [ ]:
ontime1 = ontime.dropna(subset=["arrdelay"])

ontime1["cancelled"] = ontime1["cancelled"].apply(lambda x: str(x))
ontime1 = ontime1[ontime1["cancelled"] == "0"]

ontime1["diverted"] = ontime1["diverted"].apply(lambda x: str(x))
ontime1 = ontime1[ontime1["diverted"] == "0"]

ontime1["depdelay"] = ontime1["depdelay"].apply(lambda x: 0 if x < 0 else x)
ontime1["arrdelay"] = ontime1["arrdelay"].apply(lambda x: 0 if x < 0 else x)

Group by date and compute average daily arrival delay. 

In [ ]:
avgarrdelay = ontime1.groupby(ontime["date"])["arrdelay"].mean()

Use calplot package to generate a calendar plot with data arranged in series such that date is the index and 
daily average arrival delay are the values.Use YlOrRD color palettes to demonstrate 
the length of daily arrival delay.Each rectangle of the calendar represents a day of the year. The color of each rectangle is determined by the 
length of average daily arrival delays. Shade of dark red demonstrates long average daily arrival delay whereas 
shade of light yellow depicts short average daily arrival delays.

In [ ]:
calplot.calplot(avgarrdelay, cmap = "YlOrRd", suptitle = "Average Arrival Delay from 2006 to 2007 (minutes)")
plt.show()

### b. Bar plots for average monthly arrival delay for 2006 and 2007
Group by month and calculate average arrival delay for each month.

In [ ]:
avgarrdelaymonthly = (ontime1.groupby(ontime["date"].dt.strftime("%b"))["arrdelay"].mean().sort_values(ascending = False))
avgarrdelaymonthly = pd.DataFrame({"month":avgarrdelaymonthly.index, 
                                   "monthlyavgarrdelay":avgarrdelaymonthly.values})
avgarrdelaymonthly["monthlyavgarrdelay"] = avgarrdelaymonthly["monthlyavgarrdelay"].apply(lambda x: round(x, 2))

Define a function called <code> addlabels( ) </code> for labels of bar charts.

In [ ]:
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i]+0.2, y[i], ha = "center", fontsize = 11, fontname = "sans")

Generate the bar plots of monthly average arrival delay with Paired color palettes.

In [ ]:
fig, ax = plt.subplots(figsize = (10,5))
col_map = plt.get_cmap("Paired")

ax.bar(avgarrdelaymonthly.month,avgarrdelaymonthly.monthlyavgarrdelay, color = col_map.colors, 
       edgecolor = "white",width = 0.8)
ax.set_title("Average Arrival Delay per Month", fontname = "sans", fontweight = "bold", fontsize = 20)
ax.set_xlabel("Month", fontname = "sans", fontweight = "light", fontsize = 15)
ax.set_ylabel("Average Arrival Delay (minutes)", fontname = "sans", fontweight = "light", fontsize = 15)

addlabels(avgarrdelaymonthly.month,avgarrdelaymonthly.monthlyavgarrdelay)

plt.xticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.show()

### c. Bar plots of daily average arrival delay from 2006 to 2007.

Group by day of the week and calculate average arrival delay for each day of week. Remove non-applicable values. The days of the week are Monday, Tuesday, Wednesday, Thursday, Friday, Saturday and Sunday.

In [ ]:
avgarrdelaydaily = (ontime1.groupby(ontime["date"].dt.strftime("%a"))["arrdelay"].mean().
                    sort_values(ascending = False))
avgarrdelaydaily = pd.DataFrame({"dayofweek":avgarrdelaydaily.index, 
                                 "dailyavgarrdelay":avgarrdelaydaily.values})
avgarrdelaydaily["dailyavgarrdelay"] = avgarrdelaydaily["dailyavgarrdelay"].apply(lambda x: round(x, 2))

Generate the bar plots of average arrival delay for day of week from 2006 to 2007 with Paired color palettes.

In [ ]:
fig, ax = plt.subplots(figsize = (10,5))
col_map = plt.get_cmap("Paired")

ax.bar(avgarrdelaydaily.dayofweek,avgarrdelaydaily.dailyavgarrdelay, color = col_map.colors, 
       edgecolor = "white", width = 0.8)
ax.set_title("Average Arrival Delay per Day of Week", fontname = "sans", fontweight = "bold", fontsize = 20)
ax.set_xlabel("Day of Week", fontname = "sans", fontweight = "light", fontsize = 15)
ax.set_ylabel("Average Arrival Delay (minutes)", fontname = "sans", fontweight = "light", fontsize = 15)
addlabels(avgarrdelaydaily.dayofweek,avgarrdelaydaily.dailyavgarrdelay)

plt.xticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.show()

### d. Bar plots of average departure and arrival delays for each time interval from 2006 to 2007.

Group by arrival time intervals and calculate average arrival delay for each time interval. Remove non-applicable values.

In [ ]:
avgarrdelaytime = ontime1.groupby(ontime["arrtimeintervals"])["arrdelay"].mean().sort_values(ascending = False)
avgarrdelaytime = pd.DataFrame({"arrtimeintervals":avgarrdelaytime.index, 
                                "timeavgarrdelay":avgarrdelaytime.values})
avgarrdelaytime["timeavgarrdelay"] = avgarrdelaytime["timeavgarrdelay"].apply(lambda x: round(x, 2))   

Generate the bar plots of average arrival delays of each time interval from 2006 to 2007 with Paired color palettes.

In [ ]:
fig, ax = plt.subplots(figsize = (10,5))
col_map = plt.get_cmap("Paired")

ax.bar(avgarrdelaytime.arrtimeintervals,avgarrdelaytime.timeavgarrdelay, color = col_map.colors, 
       edgecolor = "white", width = 0.8)
ax.set_title("Average Arrival Delay per Time Interval", fontname = "sans", fontweight = "bold", fontsize = 20)
ax.set_xlabel("Time Intervals", fontname = "sans", fontweight = "light", fontsize = 15)
ax.set_ylabel("Average Arrival Delay (minutes)", fontname = "sans", fontweight = "light", fontsize = 15)
addlabels(avgarrdelaytime.arrtimeintervals,avgarrdelaytime.timeavgarrdelay)

plt.xticks(fontname = "sans", fontweight = "light", fontsize = 15,
          rotation=45, ha="right", rotation_mode="anchor")
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.ylim([0, 83])
plt.show()

# 2. Do older planes suffer more delays?

### Idea: Determine age of plane with manufacturing year. Arrival delay ratio for each manufacturing year is computed and plotted with scatter plots fitted with the best fit line. Results from the linear regression model are analyzed to recognize the relationship between manufacturing year and arrival delay ratio.

### a. Scatter plots for ratios of arrival delays are generated against manufacturing years

The age of the planes is determined by manufacturing year, which is <code> year </code> column in <code>planes </code>data frame. Rename year column as <code> manufaturingyear </code>. Drop blanks and "None" in <code> manufaturingyear </code> column in planes data frame. Convert the values of <code> manufaturingyear </code> as integer. Then drop value of 0 in <code>manufacturingyear</code> .Inner join <code>planes</code> with <code>ontime1</code> where diverted and canceled flights are filtered out from <code>ontime</code> by <code>tailnum</code> for computation of arrival delay ratio based on manufacturing year.

In [ ]:
planes.rename(columns={"year":"manufacturingyear"}, inplace=True)
planes = planes[planes["manufacturingyear"].notna()]
planes = planes[planes["manufacturingyear"] != "None"]
planes["manufacturingyear"] = planes["manufacturingyear"].astype(int)
planes = planes[planes.manufacturingyear > 0] 
ageofplane = pd.merge(planes, ontime1[["arrdelay","biarrdelay","tailnum"]], on="tailnum")

Group by manufacturing year and calculate the sum of frequency of arrival delay and the sum of arrival and no arrival delay.

In [ ]:
arrdelayage = ageofplane.groupby(ageofplane["manufacturingyear"])["biarrdelay"].sum()
sumarrdelayage = ageofplane.groupby(ageofplane["manufacturingyear"])["biarrdelay"].count()

arrdelayage = pd.DataFrame({"manufacturingyear":arrdelayage.index, "biarrdelay":arrdelayage.values})
sumarrdelayage = pd.DataFrame({"manufacturingyear":sumarrdelayage.index, "sumbiarrdelay":sumarrdelayage.values})

Add a new column arrdelayratio to compute the ratio of arrival delay. Divide the number of arrival delays by sum of arrival and no arrival delay for arrival delay ratio.

In [ ]:
carrdelayage = pd.merge(arrdelayage,sumarrdelayage , on="manufacturingyear")
carrdelayage["arrdelayratio"] = carrdelayage["biarrdelay"]*1/carrdelayage["sumbiarrdelay"]
carrdelayage["arrdelayratio"]  = carrdelayage["arrdelayratio"].apply(lambda x: round(x, 2))

Scatter plot of ratio of arrival delays against manufacturing year is plotted. A line of best fit is fitted to demonstrate the relationship between of age of plane and arrival delay ratio.

In [ ]:
figsize = (12,12)
sns.lmplot(x ="manufacturingyear", y = "arrdelayratio",
           fit_reg = True, data = carrdelayage,line_kws={"color": "blue"}, ci = None, 
           markers="o", scatter_kws = {"color":"black","facecolors":"none"})
plt.title("Arrival Delay based on Age of Planes", 
          fontname = "sans", fontweight = "bold", fontsize = 20)
plt.xlabel("Manufacturing Year",fontname = "sans", fontweight = "light", fontsize = 15)
plt.ylabel("Arrival Delay Ratio",fontname = "sans", fontweight = "light", fontsize = 15)
plt.xticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.xlim([1950, 2010])
plt.show()

### b. Linear regression for demonstration of the linear relationship between age of plane and ratio of arrival delays.

Identify the linear relationship between age of the plane, indicated by manufacturing year and the ratio of arrival delays. Manufacturing year is the predictor variable whereas arrival delay ratio is the response variable.

In [ ]:
X = carrdelayage["manufacturingyear"]
y = carrdelayage["arrdelayratio"]

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

# 3. How does the number of people flying between different locations change over time?

### Idea: Investigate how the number of people change monthly from 2006 to 2007 between the destination city with highest number of inbound count and top 10 origin cities with highest number of outbound count to this destination city. This step is repeated with cities with 2nd and 3rd highest inbound count. The number of people is estimated with the number of outbound flights. Each flight is assumed to have 100 people.

To compute the number of people between cities, canceled flights are excluded. To identify the top 3 destination cities with the highest number of inbound counts, destination cities are included in <code>topinbound</code>, which is the inner join of <code>ontime2</code>, which excluded canceled flights from <code>ontime</code> and <code>airports</code> by <code>dest</code> and <code>iata</code> columns. Then, group by destination cities and compute the inbound count for each city. Retain data of the top 3 cities with the highest number of inbound count which are Chicago, Atlanta and Dallas-Fort Worth for <code>topinbound</code>.

In [ ]:
ontime2 = ontime
ontime2["cancelled"] = ontime2["cancelled"].apply(lambda x: str(x))
ontime2 = ontime2[ontime2["cancelled"] == "0"]

airports2 = airports.rename(columns={"iata":"dest"}, inplace=False)
topinbound = pd.merge(ontime2, airports2, on = "dest")
topinbound = topinbound.groupby("city")["city"].count().sort_values(ascending = False).head(3)
topinbound = pd.DataFrame({"destcity":topinbound.index, "inboundcount":topinbound.values})
topinbound

<code>destorigin</code> is set up for origin and destination data. <code>ontime2</code> is joined with <code>airports</code> by <code>dest = iata</code>. Then, repeat this step with <code>origin = iata</code>. To ensure that <code>destorigin</code> only contains data from top 3 cities with highest inbound count, create <code>destorigin2</code> which is the inner join of <code>destorigin</code> and <code>topinbound</code>. Create a column called <code>ym</code>, by concatenating the values from the <code>year</code> and <code>month</code> columns.

In [ ]:
destorigin = ontime2[["datedeptime", "datearrtime","year","month","origin",
         "dest","depdelay","arrdelay","tailnum","biarrdelay"]]
destorigin = pd.merge(destorigin, airports2, on = "dest")
destorigin.rename(columns={"city":"destcity","airport":"destairport"}, inplace =  True)

airports3 = airports.rename(columns={"iata":"origin"}, inplace=False)
airports3 = airports3[["origin","city","airport"]]
destorigin = pd.merge(destorigin, airports3, on = "origin")
destorigin.rename(columns={"city":"origincity","airport":"originairport"}, inplace =  True)
destorigin.drop(["lat","long","state"], axis=1, inplace=True)

destorigin2 = pd.merge(destorigin, topinbound, on = "destcity")
destorigin2["ym"] = destorigin2["datedeptime"].dt.strftime('%Y%m')
destorigin2["ym2"] = destorigin2["datedeptime"].dt.strftime('%Y %b')

destorigin2

Create data frames <code>Chicago ,Atlanta , Dallas</code> with <code>ym , origin and destcity </code>columns. <code>Chicago</code> data frame should only contain data where destcity is Chicago, <code>Alanta</code> data frame should only contain data where destcity is Atlanta and <code>Dallas-Fort Worth</code> data frame should only contain data where destcity is Dallas. Store the data frames in a dictionary.

In [ ]:
dict1 = {}
for DESTCITYY in topinbound["destcity"]:
    dict1[DESTCITYY] = destorigin2[destorigin2["destcity"] == DESTCITYY][["ym","ym2","origincity","destcity"]]
    print(DESTCITYY)

To identify the top 10 origin cities with highest outbound count to Chicago, group by origin cities and compute the outbound count of each city. Ensure the destination city of destorigin only contains Chicago. This step is repeated for Atlanta and Dallas-Fort Worth with a for loop. The outputs of Chicago with top 10 origin cities, Atlanta with top 10 origin cities and Dallas with top 10 origin cities are stored as 3 new dataframes in a dictionary.

In [ ]:
dict2 = {}
for DESTCITYY in topinbound["destcity"]:
    dict2[DESTCITYY] = destorigin2[destorigin2["destcity"] == DESTCITYY][["ym","ym2","origincity","destcity"]].groupby("origincity")["origincity"].count().sort_values(ascending = False).head(10)
    print(DESTCITYY)

The data frames in <code> dict2</code> have no destcity columns. Merge them with data frames in <code>dict1</code>

In [ ]:
Chicago = dict1.get("Chicago")
Chicago2 = dict2.get("Chicago")
Chicago2 = pd.DataFrame({"origincity":Chicago2.index, "outboundcount":Chicago2.values})
Chicago2 = pd.merge(Chicago, Chicago2, on = "origincity")
Chicago2 = Chicago2.sort_values(by=["outboundcount"], ascending = False)

Atlanta = dict1.get("Atlanta")
Atlanta2 = dict2.get("Atlanta")
Atlanta2 = pd.DataFrame({"origincity":Atlanta2.index, "outboundcount":Atlanta2.values})
Atlanta2 = pd.merge(Atlanta, Atlanta2, on = "origincity")
Atlanta2 = Atlanta2.sort_values(by=["outboundcount"], ascending = False)

Dallas = dict1.get("Dallas-Fort Worth")
Dallas2 = dict2.get("Dallas-Fort Worth")
Dallas2 = pd.DataFrame({"origincity":Dallas2.index, "outboundcount":Dallas2.values})
Dallas2 = pd.merge(Dallas, Dallas2, on = "origincity")
Dallas2 = Dallas2.sort_values(by=["outboundcount"], ascending = False)

To identify the monthly outbound count of each 10 origin cities to Chicago, group by destination cities, origin cities and ym for the computation of outbound count. The outbound count is then multiplied by 100 to represent the number of people. Repeat these step for Atlanta and Dallas. 

In [ ]:
Chicago3 = Chicago2.groupby(["destcity","origincity","ym","ym2"]).size().reset_index()
Chicago3.rename(columns={0:"outboundcount"}, inplace=True)
Chicago3["peoplecount"] = Chicago3["outboundcount"]*100
Chicago4 = (Chicago3.pivot_table(index="origincity", columns=["ym","ym2"], values="peoplecount",
               aggfunc="sum", margins=True).sort_values("All", ascending=False).drop("All", axis=1).drop("All"))  

Atlanta3 = Atlanta2.groupby(["destcity","origincity","ym","ym2"]).size().reset_index()
Atlanta3.rename(columns={0:"outboundcount"}, inplace=True)
Atlanta3["peoplecount"] = Atlanta3["outboundcount"]*100
Atlanta4 = (Atlanta3.pivot_table(index="origincity", columns=["ym","ym2"], values="peoplecount",
               aggfunc="sum", margins=True).sort_values("All", ascending=False).drop("All", axis=1).drop("All"))

Dallas3 = Dallas2.groupby(["destcity","origincity","ym","ym2"]).size().reset_index()
Dallas3.rename(columns={0:"outboundcount"}, inplace=True)
Dallas3["peoplecount"] = Dallas3["outboundcount"]*100
Dallas4 = (Dallas3.pivot_table(index="origincity", columns=["ym","ym2"], values="peoplecount",
               aggfunc="sum", margins=True).sort_values("All", ascending=False).drop("All", axis=1).drop("All"))

Plot a heat map to depicts the number of people of each month of the year flying from top 10 origin cities to Chicago, Atlanta and Dallas-Fort Worth respectively from 2006 to 2007. Each rectangle demonstrates the number of people. YlGn color palettes is chosen for the heat map. Dark green represent higher number of people whereas light yellow illustrates lower number of people.

In [ ]:
figsize = (20,8)
ymlist = Chicago3.ym2.unique().tolist()
ax = sns.heatmap(Chicago4,cmap = "YlGn", xticklabels = ymlist, cbar_kws={"label": "Number of People"},
                 yticklabels= True)
ax.set_xlabel("Period", fontname = "sans", fontweight = "light",fontsize=10)
ax.set_ylabel("Origin Cities", fontname = "sans", fontweight = "light",fontsize=10)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontname = "sans", fontweight = "light", fontsize = 10)
ax.set_yticklabels(ax.get_ymajorticklabels() , fontname = "sans", fontweight = "light", fontsize = 10)
ax.figure.axes[-1].yaxis.label.set_size(10)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize= 10)
ax.set_title("Number of people from origin cities from 2006 to 2007 to \nChicago", 
           fontname = "sans", fontweight = "bold", fontsize = 12)
plt.show()

In [ ]:
figsize = (20,8)
ymlist = Atlanta3.ym2.unique().tolist()
ax = sns.heatmap(Atlanta4,cmap = "YlGn", xticklabels = ymlist, cbar_kws={"label": "Number of People"}, 
                 yticklabels= True)
ax.set_xlabel("Period", fontname = "sans", fontweight = "light",fontsize=10)
ax.set_ylabel("Origin Cities", fontname = "sans", fontweight = "light",fontsize=10)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontname = "sans", fontweight = "light", fontsize = 10)
ax.set_yticklabels(ax.get_ymajorticklabels() , fontname = "sans", fontweight = "light", fontsize = 10)
ax.figure.axes[-1].yaxis.label.set_size(10)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize= 10)
ax.set_title("Number of people from origin cities from 2006 to 2007 to \n Atlanta", 
           fontname = "sans", fontweight = "bold", fontsize = 12)
plt.show()

In [ ]:
figsize = (20,8)
ymlist = Dallas3.ym2.unique().tolist()
ax = sns.heatmap(Dallas4,cmap = "YlGn", xticklabels = ymlist, cbar_kws={"label": "Number of People"}, 
                 yticklabels= True)
ax.set_xlabel("Period", fontname = "sans", fontweight = "light",fontsize=10)
ax.set_ylabel("Origin Cities", fontname = "sans", fontweight = "light",fontsize=10)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontname = "sans", fontweight = "light", fontsize = 10)
ax.set_yticklabels(ax.get_ymajorticklabels() , fontname = "sans", fontweight = "light", fontsize = 10)
ax.figure.axes[-1].yaxis.label.set_size(10)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize= 10)
ax.set_title("Number of people from origin cities from 2006 to 2007 to\n Dallas", 
           fontname = "sans", fontweight = "bold", fontsize = 12)
plt.show()

# 4. Can you detect cascading failures as delays in one airport create delays in others?

### Idea: Study how departure delay of planes from an airport led to cascading arrival and departure delays of planes to other airports on the day where average arrival delay is the longest.

To identify cascading failures, canceled and diverted flights are excluded. For origin and destination airports data, <code> destorigin3</code> is set up. Join <code>ontime1</code> from <code>ontime</code> which excludes canceled and diverted flights with <code>airports2 by <code>dest = iata </code>. Then, repeat this step with origin = iata. 

In [ ]:
destorigin3 = ontime1[["datedeptime", "datearrtime","datecrsdeptime", "datecrsarrtime", "date",
         "origin","dest","depdelay","arrdelay", "tailnum","flightnum","biarrdelay"]]
destorigin3 = pd.merge(destorigin3, airports2, on = "dest")
destorigin3.rename(columns={"city":"destcity","airport":"destairport"}, inplace =  True)

destorigin3 = pd.merge(destorigin3, airports3, on = "origin")
destorigin3 = destorigin3[["datedeptime", "datearrtime","datecrsdeptime", "datecrsarrtime", "date",
                           "origin","dest","city","destcity","airport","destairport","depdelay","arrdelay", 
                           "tailnum","flightnum","biarrdelay"]]
destorigin3.rename(columns={"city":"origincity","airport":"originairport"}, inplace =  True)

destorigin3

Ensure that <code> destorigin4 </code> only contains the data from the date with the longest average arrival delay, which is on 2006-01-02. Arrange the data by tail number and actual departure time in ascending order. Ensure that arrival time is later than departure time, departure time is later than previous arrival time.

In [ ]:
maxavgarrdelay = avgarrdelay.nlargest(1)
maxavgarrdelay = pd.DataFrame({"date":maxavgarrdelay.index, "avgarrdelay":maxavgarrdelay.values})
maxavgarrdelay = pd.DataFrame(np.repeat(maxavgarrdelay.values, destorigin3.shape[0], axis = 0))
maxavgarrdelay.rename(columns = {0:"date" ,1:"avgarrdelay"}, inplace = True)

destorigin4 = destorigin3[destorigin3["date"] == maxavgarrdelay.date]

destorigin4 = destorigin4[["date","tailnum","flightnum","datecrsdeptime","datecrsarrtime",
                          "datedeptime", "datearrtime", "originairport", "destairport",
                          "depdelay","arrdelay"]].sort_values(["tailnum","datedeptime"], ascending = True)

destorigin4 = destorigin4[destorigin4["datearrtime"] > destorigin4["datedeptime"]]

destorigin4["lagdatearrtime"] = destorigin4["datearrtime"].shift(1)

destorigin4 = destorigin4[destorigin4["datedeptime"] > destorigin4["lagdatearrtime"]]

destorigin4

Select top 6 planes with highest number of flights on 2006-01-02. The top plane has 13 flights whereas the 2nd to 6th planes have 12 flights. 

In [ ]:
destorigin5 = destorigin4.groupby(destorigin4["tailnum"])["arrdelay"].count().sort_values(ascending = False).head(6)
destorigin5 = pd.DataFrame({"tailnum":destorigin5.index, 
                                "frequency":destorigin5.values})
destorigin5

Select data with only the top 6 planes.Filter the data such that only data with departure and arrival delay are selected.

In [ ]:
toptailnum = pd.merge(destorigin4, destorigin5, on = "tailnum")
toptailnum2 = toptailnum[(toptailnum.depdelay > 0) & (toptailnum.arrdelay > 0)]
toptailnum2

Create a dictionary for original data and another dictionary for data with departure and arrival delay. Each data frame in the dictionary contains data for each unique tail number.

In [ ]:
toptailnumdict = {}
for NUM in toptailnum["tailnum"]:
    toptailnumdict[NUM] = toptailnum[toptailnum["tailnum"] == NUM]
    
toptailnumdict2 = {}
for NUM in toptailnum2["tailnum"]:
    toptailnumdict2[NUM] = toptailnum2[toptailnum2["tailnum"] == NUM] 

Identify planes with departure and arrival delay.

In [ ]:
toptailnumdict2.keys()

To study flights with cascading departure and arrival delay, compare the index of original data frames with filtered data frame and remove all rows below if there are missing index in the middle of the data frames.Row with missing index implies that plane did not have delay for that timing, hence breaking the cascading effect. The missing indexes are not in the middle of the data frames. Hence, no rows are needed to be removed.

In [ ]:
N226SW_2 = toptailnumdict2.get("N226SW")
N271YV_2 = toptailnumdict2.get("N271YV")
N480HA_2 = toptailnumdict2.get("N480HA")
N226SW_1 = toptailnumdict.get("N226SW")
N271YV_1 = toptailnumdict.get("N271YV")
N480HA_1 = toptailnumdict.get("N480HA")

index1 = N226SW_1.index.difference(N226SW_2.index)
index2 = N271YV_1.index.difference(N271YV_2.index)
index3 = N480HA_1.index.difference(N480HA_2.index)
print(index1,index2,index3)

N226SW had more than 3 consecutive departure and arrival delays respectively.

In [ ]:
N226SW_2 = N226SW_2[["tailnum", "flightnum","datecrsdeptime","datedeptime","datecrsarrtime",
         "datearrtime","originairport","destairport"]].reset_index() 
N226SW_2 = N226SW_2.iloc[: , 1:]
N226SW_2 

N271YV had more than 3 consecutive departure and arrival delays respectively.

In [ ]:
N271YV_2 = N271YV_2[["tailnum", "flightnum","datecrsdeptime","datedeptime","datecrsarrtime",
         "datearrtime","originairport","destairport"]].reset_index() 
N271YV_2 = N271YV_2.iloc[: , 1:]
N271YV_2

N480HA has less than 3 consecutive departure and arrival delays respectively. Hence, it is not selected for the study of cascading failure.

In [ ]:
N480HA_2 = N480HA_2[["tailnum", "flightnum","datecrsdeptime","datedeptime","datecrsarrtime",
         "datearrtime","originairport","destairport"]].reset_index() 
N480HA_2 = N480HA_2.iloc[: , 1:]
N480HA_2

Create edge list for N226SW.Group by edges to compute the number of connections between 2 nodes. Store the output of the number of connections in a new column called <code> weight </code>

In [ ]:
N226SW_2["edges"] = N226SW_2[["originairport","destairport"]].apply(tuple, axis = 1)
N226SW_3 = N226SW_2.groupby(["edges"]).size().sort_values(ascending = False)
N226SW_3 = pd.DataFrame({"edges":N226SW_3 .index, "weight":N226SW_3 .values})
N226SW_3["originairport"], N226SW_3["destairport"] = N226SW_3.edges.str
N226SW_3 = N226SW_3[["originairport","destairport","weight"]]
N226SW_3

Create a directed graph for cascading failures of N226SW on 2006-01-02.

In [ ]:
G = nx.DiGraph()  

for index, row in N226SW_3.iterrows():
    G.add_edge(row["originairport"], row["destairport"], weight=row["weight"], color = "#808080")
  
remove = [node for node,degree in G.degree() if degree ==0]
G.remove_nodes_from(remove)

colors = nx.get_edge_attributes(G,"color").values()

weights = nx.get_edge_attributes(G,"weight").values()
weights = np.divide(list(weights),(1/3))

options = {
     "node_color": "#f03b20",
     "alpha": 1,
     "connectionstyle": "arc3, rad=0.1"} 

plt.subplots(figsize=(10,5))

pos=nx.spring_layout(G)

d = dict(G.degree(weight="weight"))

nx.draw(G, pos=pos, 
        nodelist=list(d.keys()), 
        node_size=[v*1000 for v in d.values()],  
        width=weights, edge_color=colors, **options)
nx.draw_networkx_labels(G, pos=pos, font_size = 12)

plt.tight_layout()
plt.axis('off')
plt.title("Cascading failures of N226SW on 2006-01-02", 
          fontname = "sans", fontweight = "bold", fontsize = 20)
plt.show()

Create edge list for N271YV. Group by edges to compute the number of connections between 2 nodes. Store the output of the number of connections in a new column called <code> weight </code>

In [ ]:
N271YV_2["edges"] = N271YV_2[["originairport","destairport"]].apply(tuple, axis = 1)
N271YV_3 = N271YV_2.groupby(["edges"]).size().sort_values(ascending = False)
N271YV_3 = pd.DataFrame({"edges":N271YV_3 .index, "weight":N271YV_3 .values})
N271YV_3["originairport"], N271YV_3["destairport"] = N271YV_3.edges.str
N271YV_3 = N271YV_3[["originairport","destairport","weight"]]
N271YV_3

Create a directed graph for cascading failures of N271YV on 2006-01-02.

In [ ]:
G2 = nx.DiGraph()  

for index, row in N271YV_3.iterrows():
    G2.add_edge(row["originairport"], row["destairport"], weight=row["weight"], color = "#808080")
  
remove = [node for node,degree in G2.degree() if degree ==0]
G2.remove_nodes_from(remove)

colors = nx.get_edge_attributes(G2,"color").values()

weights = nx.get_edge_attributes(G2,"weight").values()
weights = np.divide(list(weights),(1/5))

options = {
     "node_color": "#fecc5c",
     "alpha": 1,
     "connectionstyle": "arc3, rad=0.1"} 

plt.subplots(figsize=(10,5))

pos=nx.spring_layout(G2)

d = dict(G2.degree(weight="weight"))

nx.draw(G2, pos=pos, 
        nodelist=list(d.keys()), 
        node_size=[v*850 for v in d.values()],  
        width=weights, edge_color=colors, **options)
nx.draw_networkx_labels(G2, pos=pos, font_size = 12)

plt.tight_layout()
plt.axis('off')
plt.title("Cascading failures of N271YV on 2006-01-02", 
          fontname = "sans", fontweight = "bold", fontsize = 20)
plt.show()

# 5. Use available variables to construct a model that predicts delays

### Idea: A target variable is chosen and exploratory data analysis is conducted for features selection. Data is sampled to process datasets. Different classification models are trained with training sets and their performance is evaluated by running the trained models on test sets. Grid search with cross validations are conducted using training set for hyperparameter optimisation. ROC curves are used as a measure to compare performance of the classification models. 

### a. Exploratory data analysis

Explore the relationship between scheduled departure time and arrival delay ratio. Compute arrival delay ratio for each scheduled departure time interval.

In [ ]:
arrdelaydeptime = ontime1.groupby(ontime1["crsdeptimeintervals"])["biarrdelay"].sum()
sumarrdelaydeptime = ontime1.groupby(ontime1["crsdeptimeintervals"])["biarrdelay"].count()

arrdelaydeptime = pd.DataFrame({"crsdeptimeintervals":arrdelaydeptime.index, "biarrdelay":arrdelaydeptime.values})
sumarrdelaydeptime = pd.DataFrame({"crsdeptimeintervals":sumarrdelaydeptime.index, "sumbiarrdelay":sumarrdelaydeptime.values})

carrdelaydeptime = pd.merge(arrdelaydeptime, sumarrdelaydeptime , on="crsdeptimeintervals")
carrdelaydeptime["arrdelayratio"] = carrdelaydeptime["biarrdelay"]*1/carrdelaydeptime["sumbiarrdelay"]
carrdelaydeptime["arrdelayratio"]  = carrdelaydeptime["arrdelayratio"].apply(lambda x: round(x, 2))
carrdelaydeptime = carrdelaydeptime.sort_values("arrdelayratio", ascending = False)

In [ ]:
fig, ax = plt.subplots(figsize = (12,9))
col_map = plt.get_cmap("Paired")

ax.bar(carrdelaydeptime.crsdeptimeintervals,carrdelaydeptime.arrdelayratio, color = col_map.colors, 
       edgecolor = "white", width = 0.8)
ax.set_title("Arrival Delay Ratio based on \n Scheduled Departure Time Intervals", fontname = "sans",
             fontweight = "bold", fontsize = 20)
ax.set_xlabel("Scheduled Departure Time Intervals", fontname = "sans", fontweight = "bold", fontsize = 15)
ax.set_ylabel("Arrival Delay Ratio", fontname = "sans", fontweight = "bold", fontsize = 15)

plt.xticks(rotation = 45, ha="right", rotation_mode="anchor",
          fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.show()

Explore the relationship between scheduled arrival time and arrival delay ratio. Compute arrival delay ratio for each scheduled arrival time interval.

In [ ]:
arrdelayarrtime = ontime1.groupby(ontime1["crsarrtimeintervals"])["biarrdelay"].sum()
sumarrdelayarrtime = ontime1.groupby(ontime1["crsarrtimeintervals"])["biarrdelay"].count()

arrdelayarrtime = pd.DataFrame({"crsarrtimeintervals":arrdelayarrtime.index, "biarrdelay":arrdelayarrtime.values})
sumarrdelayarrtime = pd.DataFrame({"crsarrtimeintervals":sumarrdelayarrtime.index, "sumbiarrdelay":sumarrdelayarrtime.values})

carrdelayarrtime = pd.merge(arrdelayarrtime, sumarrdelayarrtime , on = "crsarrtimeintervals")
carrdelayarrtime["arrdelayratio"] = carrdelayarrtime["biarrdelay"]*1/carrdelayarrtime["sumbiarrdelay"]
carrdelayarrtime["arrdelayratio"]  = carrdelayarrtime["arrdelayratio"].apply(lambda x: round(x, 2))
carrdelayarrtime = carrdelayarrtime.sort_values("arrdelayratio", ascending = False)

In [ ]:
fig, ax = plt.subplots(figsize = (12,9))
col_map = plt.get_cmap("Paired")

ax.bar(carrdelayarrtime.crsarrtimeintervals,carrdelayarrtime.arrdelayratio, color = col_map.colors, 
       edgecolor = "white", width = 0.8)
ax.set_title("Arrival Delay Ratio \nbased on Scheduled Arrival Time Intervals", fontname = "sans", 
             fontweight = "bold", 
             fontsize = 20)
ax.set_xlabel("Scheduled Arrival Time Intervals", fontname = "sans", fontweight = "bold", fontsize = 15)
ax.set_ylabel("Arrival Delay Ratio", fontname = "sans", fontweight = "bold", fontsize = 15)

plt.xticks(rotation = 45, ha="right", rotation_mode="anchor",
          fontname = "sans", fontweight = "light", fontsize = 15)
plt.xticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.show()

Explore the relationship between carrier and arrival delay ratio. Compute arrival delay ratio for each carrier.

In [ ]:
carriers = carriers.rename(columns = str.lower)
carriers.rename(columns = {"code":"uniquecarrier"}, inplace = True)

carrierdelay = pd.merge(ontime1, carriers, on = "uniquecarrier")

arrdelaycarrier = carrierdelay.groupby(carrierdelay["description"])["biarrdelay"].sum()
sumarrdelaycarrier = carrierdelay.groupby(carrierdelay["description"])["biarrdelay"].count()

arrdelaycarrier = pd.DataFrame({"carrier":arrdelaycarrier.index, "biarrdelay":arrdelaycarrier.values})
sumarrdelaycarrier = pd.DataFrame({"carrier":sumarrdelaycarrier.index, "sumbiarrdelay":sumarrdelaycarrier.values})

carrdelaycarrier = pd.merge(arrdelaycarrier, sumarrdelaycarrier , on="carrier")
carrdelaycarrier["arrdelayratio"] = carrdelaycarrier["biarrdelay"]*1/carrdelaycarrier["sumbiarrdelay"]
carrdelaycarrier["arrdelayratio"]  = carrdelaycarrier["arrdelayratio"].apply(lambda x: round(x, 2))

carrdelaycarrier2 = carrdelaycarrier.sort_values("arrdelayratio", ascending = False).head(10)

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
col_map = plt.get_cmap("Paired")

ax.bar(carrdelaycarrier2.carrier,carrdelaycarrier2.arrdelayratio, color = col_map.colors, 
       edgecolor = "white", width = 0.8)
ax.set_title("Arrival Delay Ratio based on Carriers", fontname = "sans", fontweight = "bold", fontsize = 20)
ax.set_xlabel("Carriers", fontname = "sans", fontweight = "bold", fontsize = 15)
ax.set_ylabel("Arrival Delay Ratio", fontname = "sans", fontweight = "bold", fontsize = 15)


plt.xticks(rotation = 45, ha="right", rotation_mode="anchor",
          fontname = "sans", fontweight = "light", fontsize = 15)
plt.xticks(rotation=45, ha="right", rotation_mode="anchor")
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.show()

Explore the relationship between destination airport and arrival delay ratio. Compute arrival delay ratio for each destination airport.

In [ ]:
arrdelayda = destorigin3.groupby(destorigin3["destairport"])["biarrdelay"].sum()
sumarrdelayda = destorigin3.groupby(destorigin3["destairport"])["biarrdelay"].count()

arrdelayda = pd.DataFrame({"destairport":arrdelayda.index, "biarrdelay":arrdelayda.values})
sumarrdelayda = pd.DataFrame({"destairport":sumarrdelayda.index, "sumbiarrdelay":sumarrdelayda.values})

carrdelayda = pd.merge(arrdelayda, sumarrdelayda , on = "destairport")
carrdelayda["arrdelayratio"] = carrdelayda["biarrdelay"]*1/carrdelayda["sumbiarrdelay"]
carrdelayda["arrdelayratio"]  = carrdelayda["arrdelayratio"].apply(lambda x: round(x, 2))

carrdelayda2 = carrdelayda.sort_values("arrdelayratio", ascending = False).head(10)

In [ ]:
fig, ax = plt.subplots(figsize = (12,9))
col_map = plt.get_cmap("Paired")

ax.bar(carrdelayda2.destairport,carrdelayda2.arrdelayratio, color = col_map.colors, 
       edgecolor = "white", width = 0.8)
ax.set_title("Arrival Delay Ratio based on Destination Airport", fontname = "sans", fontweight = "bold", 
             fontsize = 20)
ax.set_xlabel("Destination Airport", fontname = "sans", fontweight = "bold", fontsize = 15)
ax.set_ylabel("Arrival Delay Ratio", fontname = "sans", fontweight = "bold", fontsize = 15)


plt.xticks(rotation = 45, ha="right", rotation_mode="anchor",
          fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.show()

Explore the relationship between origin airport and arrival delay ratio. Compute arrival delay ratio for each origin airport.

In [ ]:
arrdelayoa = destorigin3.groupby(destorigin3["originairport"])["biarrdelay"].sum()
sumarrdelayoa = destorigin3.groupby(destorigin3["originairport"])["biarrdelay"].count()

arrdelayoa = pd.DataFrame({"originairport":arrdelayoa.index, "biarrdelay":arrdelayoa.values})
sumarrdelayoa = pd.DataFrame({"originairport":sumarrdelayoa.index, "sumbiarrdelay":sumarrdelayoa.values})

carrdelayoa = pd.merge(arrdelayoa, sumarrdelayoa , on = "originairport")
carrdelayoa["arrdelayratio"] = carrdelayoa["biarrdelay"]*1/carrdelayoa["sumbiarrdelay"]
carrdelayoa["arrdelayratio"]  = carrdelayoa["arrdelayratio"].apply(lambda x: round(x, 2))

carrdelayoa2 = carrdelayoa.sort_values("arrdelayratio", ascending = False).head(10)

In [ ]:
fig, ax = plt.subplots(figsize = (12,9))
col_map = plt.get_cmap("Paired")

ax.bar(carrdelayoa2.originairport,carrdelayoa2.arrdelayratio, color = col_map.colors, 
       edgecolor = "white", width = 0.8)
ax.set_title("Arrival Delay Ratio based on Origin Airport", fontname = "sans", fontweight = "bold", 
             fontsize = 20)
ax.set_xlabel("Origin Airport", fontname = "sans", fontweight = "bold", fontsize = 15)
ax.set_ylabel("Arrival Delay Ratio", fontname = "sans", fontweight = "bold", fontsize = 15)


plt.xticks(rotation = 45, ha="right", rotation_mode="anchor",
          fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.show()

Explore the relationship between month and arrival delay ratio. Compute arrival delay ratio for each month.

In [ ]:
ontime1["month2"] = ontime1["month"].astype(int)
ontime1["month2"] = ontime1["month2"].apply(lambda x: calendar.month_abbr[x])

arrdelaymonth= ontime1.groupby(ontime1["month2"])["biarrdelay"].sum()
sumarrdelaymonth = ontime1.groupby(ontime1["month2"])["biarrdelay"].count()

arrdelaymonth = pd.DataFrame({"month":arrdelaymonth.index, "biarrdelay":arrdelaymonth.values})
sumarrdelaymonth = pd.DataFrame({"month":sumarrdelaymonth.index, "sumbiarrdelay":sumarrdelaymonth.values})

carrdelaymonth = pd.merge(arrdelaymonth, sumarrdelaymonth , on = "month")
carrdelaymonth["arrdelayratio"] = carrdelaymonth["biarrdelay"]*1/carrdelaymonth["sumbiarrdelay"]
carrdelaymonth["arrdelayratio"]  = carrdelaymonth["arrdelayratio"].apply(lambda x: round(x, 2))
carrdelaymonth = carrdelaymonth.sort_values("arrdelayratio", ascending = False)

In [ ]:
fig, ax = plt.subplots(figsize = (12,9))
col_map = plt.get_cmap("Paired")

ax.bar(carrdelaymonth.month,carrdelaymonth.arrdelayratio, color = col_map.colors, 
       edgecolor = "white", width = 0.8)
ax.set_title("Arrival Delay Ratio based on Month", fontname = "sans", fontweight = "bold", fontsize = 20)
ax.set_xlabel("Month", fontname = "sans", fontweight = "bold", fontsize = 15)
ax.set_ylabel("Arrival Delay Ratio", fontname = "sans", fontweight = "bold", fontsize = 15)

plt.xticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.show()

Explore the relationship between day of week and arrival delay ratio. Compute arrival delay ratio for each day of week.

In [ ]:
ontime1["dayofweek2"] = ontime1["dayofweek"].astype(int)
ontime1["dayofweek2"] = ontime1["dayofweek2"]-1
ontime1["dayofweek2"] = ontime1["dayofweek2"].apply(lambda x: calendar.day_abbr[x])

arrdelaydow = ontime1.groupby(ontime1["dayofweek2"])["biarrdelay"].sum()
sumarrdelaydow  = ontime1.groupby(ontime1["dayofweek2"])["biarrdelay"].count()

arrdelaydow = pd.DataFrame({"dayofweek":arrdelaydow.index, "biarrdelay":arrdelaydow.values})
sumarrdelaydow = pd.DataFrame({"dayofweek":sumarrdelaydow.index, "sumbiarrdelay":sumarrdelaydow.values})

carrdelaydow = pd.merge(arrdelaydow , sumarrdelaydow , on="dayofweek")
carrdelaydow["arrdelayratio"] = carrdelaydow["biarrdelay"]*1/carrdelaydow["sumbiarrdelay"]
carrdelaydow["arrdelayratio"]  = carrdelaydow["arrdelayratio"].apply(lambda x: round(x, 2))
carrdelaydow = carrdelaydow.sort_values("arrdelayratio", ascending = False)

In [ ]:
fig, ax = plt.subplots(figsize = (12,9))
col_map = plt.get_cmap("Paired")

ax.bar(carrdelaydow.dayofweek,carrdelaydow.arrdelayratio, color = col_map.colors, 
       edgecolor = "white", width = 0.8)
ax.set_title("Arrival Delay Ratio based on Day of Week", fontname = "sans", fontweight = "bold", fontsize = 20)
ax.set_xlabel("Day of Week", fontname = "sans", fontweight = "bold", fontsize = 15)
ax.set_ylabel("Arrival Delay Ratio", fontname = "sans", fontweight = "bold", fontsize = 15)
# addlabels(carrdelaycarrier2.carrier,carrdelaycarrier2.arrdelayratio)

plt.xticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.show()

### b. Target variable and features selection

The target variable is <code>biarrdelay</code>. If the arrival delay, <code>arrdelay</code> is more than 0, <code>biarrdelay</code> = 1. Else, <code>biarrdelay</code> = 0. Based on exploratory data analysis, the features such as <code>crsdeptimeintervals, crsarrtimeintervals, description, destairport, originairport, month, dayofweek</code> are chosen as they have influence on the outcome of <code>biarrdelay</code>. 

A dataframe, <code>ontime3</code> is set up by filtering out canceled and diverted flights from ontime and containing all the features and target variable needed for predictive modelling.

In [ ]:
ontime3 = ontime[ontime["cancelled"] == 0]
ontime3 = ontime[ontime["diverted"] == 0]
ontime3 = pd.merge(ontime3, airports2, on = "dest")
ontime3 = pd.merge(ontime3, carriers, on = "uniquecarrier")
ontime3 = ontime3[["biarrdelay","crsdeptimeintervals","crsarrtimeintervals", "description", "airport",
                   "month", "dayofweek", "origin"]]
ontime3.rename(columns = {"description": "carrier", "airport": "destairport"}, inplace = True)

ontime3 = pd.merge(ontime3, airports3, on = "origin")
ontime3 = ontime3[["biarrdelay","crsdeptimeintervals","crsarrtimeintervals", "carrier", "airport","destairport", 
                   "month", "dayofweek"]]
ontime3.rename(columns = {"description": "carrier", "airport": "originairport"}, inplace = True)

In [ ]:
ontime3

### c. Sampled data from original data

The data from 2006 to 2007 of approximately 14 million rows. 20% of the data is sampled such that the distribution of the all variables of the sampled data is the same with original data.

In [ ]:
ontime4 = ontime3.sample(int(len(ontime3)*0.2))
ontime4

Add new column in <code>ontime3</code> and <code>ontime4</code> with character "original" and "sample" respectively. Bind them by rows.

In [ ]:
ontime3["char"] = "original"
ontime4["char"] = "sample"
ontime4b = ontime3.append(ontime4)
ontime4b

Plot the distribution of original versus sampled data for scheduled departure time intervals.

In [ ]:
counts = ontime4b.groupby(["crsdeptimeintervals","char"]).count()
counts = counts.iloc[:,0:1]

totals = counts.sum(level=0)

counts = counts.unstack(level=1)
counts.columns = counts.columns.droplevel(level=0)
counts

fig, ax = plt.subplots(figsize = (12,9))
crsdeptimeintervals = ontime4b.crsdeptimeintervals.unique()
plt.bar(crsdeptimeintervals, counts["sample"], bottom=None, color= "#1F78B4", label= "sample")
plt.bar(crsdeptimeintervals, counts["original"], bottom = counts["sample"], color="#A6CEE3", label= "original")
plt.legend()
plt.xticks(rotation = 45, ha="right", rotation_mode="anchor",
          fontname = "sans", fontweight = "light", fontsize = 15)
plt.xlabel("Scheduled Departure Time Intervals",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.ylabel("Count",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.title("Distribution of Original versus Sampled Data",
          fontname = "sans", fontweight = "bold", fontsize = 15)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25),
          fancybox = False, shadow = False, fontsize = 15)
plt.show()

Plot the distribution of original versus sampled data for scheduled arrival time intervals.

In [ ]:
counts2 = ontime4b.groupby(["crsarrtimeintervals","char"]).count()
counts2 = counts2.iloc[:,0:1]

totals2 = counts2.sum(level=0)

counts2 = counts2.unstack(level=1)
counts2.columns = counts2.columns.droplevel(level=0)
counts2

fig, ax = plt.subplots(figsize = (12,9))
crsarrtimeintervals = ontime4b.crsarrtimeintervals.unique().astype(str)
plt.bar(crsarrtimeintervals ,counts2["sample"], bottom = None, color= "#1F78B4", label= "sample")
plt.bar(crsarrtimeintervals, counts2["original"], bottom = counts2["sample"], color="#A6CEE3", label= "original")
plt.legend()
plt.xlabel("Scheduled Arrival Time Intervals",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.ylabel("Count",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.title("Distribution of Original versus Sampled Data",
          fontname = "sans", fontweight = "bold", fontsize = 15)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25),
          fancybox = False, shadow = False, fontsize = 15)
plt.show()

Plot the distribution of original versus sampled data for carrier.

In [ ]:
counts3 = ontime4b.groupby(["carrier","char"]).count()
counts3 = counts3.iloc[:,0:1]

totals3 = counts3.sum(level=0)

counts3 = counts3.unstack(level=1)
counts3.columns = counts3.columns.droplevel(level=0)
counts3


fig, ax = plt.subplots(figsize = (12,9))
carrier = ontime4b.carrier.unique()
plt.bar(carrier, counts3["sample"], bottom=None, color= "#1F78B4", label= "sample")
plt.bar(carrier, counts3["original"], bottom = counts3["sample"], color="#A6CEE3", label= "original")
plt.legend()
plt.ylabel("Count",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.title("Distribution of Original versus Sampled Data (Carrier)",
          fontname = "sans", fontweight = "bold", fontsize = 15)
ax.axes.xaxis.set_visible(False)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox = False, shadow = False, fontsize = 15)
plt.show()

Plot the distribution of original versus sampled data for destination airport.

In [ ]:
counts4 = ontime4b.groupby(["destairport","char"]).count()
counts4 = counts4.iloc[:,0:1]

totals4 = counts4.sum(level=0)

counts4 = counts4.unstack(level=1)
counts4.columns = counts4.columns.droplevel(level=0)
counts4


fig, ax = plt.subplots(figsize = (12,9))
destairport = ontime4b.destairport.unique()
plt.bar(destairport, counts4["sample"], bottom=None, color= "#1F78B4", label= "sample")
plt.bar(destairport, counts4["original"], bottom = counts4["sample"], color="#A6CEE3", label= "original")
plt.legend()
plt.ylabel("Count",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.title("Distribution of Original versus Sampled Data (Destination Airport)",
          fontname = "sans", fontweight = "bold", fontsize = 15)
ax.axes.xaxis.set_visible(False)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox = False, shadow = False, fontsize = 15)
plt.show()

Plot the distribution of original versus sampled data for origin airport.

In [ ]:
counts4b = ontime4b.groupby(["originairport","char"]).count()
counts4b = counts4b.iloc[:,0:1]

totals4b = counts4b.sum(level=0)

counts4b = counts4b.unstack(level=1)
counts4b.columns = counts4b.columns.droplevel(level=0)
counts4b


fig, ax = plt.subplots(figsize = (12,9))
originairport = ontime4b.originairport.unique()
plt.bar(originairport, counts4b["sample"], bottom=None, color= "#1F78B4", label= "sample")
plt.bar(originairport, counts4b["original"], bottom = counts4b["sample"], color="#A6CEE3", label= "original")
plt.legend()
plt.ylabel("Count",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.title("Distribution of Original versus Sampled Data (Origin Airport)",
          fontname = "sans", fontweight = "bold", fontsize = 15)
ax.axes.xaxis.set_visible(False)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox = False, shadow = False, fontsize = 15)
plt.show()

Plot the distribution of original versus sampled data for month.

In [ ]:
counts5 = ontime4b.groupby(["month","char"]).count()
counts5 = counts5.iloc[:,0:1]

totals5 = counts5.sum(level=0)

counts5 = counts5.unstack(level=1)
counts5.columns = counts5.columns.droplevel(level=0)
counts5


fig, ax = plt.subplots(figsize = (12,9))
month = ontime4b.month.unique()
plt.bar(month, counts5["sample"], bottom=None, color= "#1F78B4", label= "sample")
plt.bar(month, counts5["original"], bottom = counts5["sample"], color="#A6CEE3", label= "original")
plt.legend()
plt.xlabel("Month",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.ylabel("Count",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.title("Distribution of Original versus Sampled Data",
          fontname = "sans", fontweight = "bold", fontsize = 15)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox = False, shadow = False)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox = False, shadow = False, fontsize = 15)
month2 = ["Jan","Feb","Mar","April","May","June","July","Aug","Sept","Oct","Nov","Dec"]
ax.set_xticklabels(month2, minor=False, rotation=0)
plt.show()

Plot the distribution of original versus sampled data for day of week.

In [ ]:
counts6 = ontime4b.groupby(["dayofweek","char"]).count()
counts6 = counts6.iloc[:,0:1]

totals6 = counts6.sum(level=0)

counts6 = counts6.unstack(level=1)
counts6.columns = counts6.columns.droplevel(level=0)
counts6


fig, ax = plt.subplots(figsize = (12,9))
dayofweek = ontime4b.dayofweek.unique()
plt.bar(dayofweek, counts6["sample"], bottom=None, color= "#1F78B4", label= "sample")
plt.bar(dayofweek, counts6["original"], bottom = counts6["sample"], color="#A6CEE3", label= "original")
plt.legend()
plt.xlabel("Day of Week",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.ylabel("Count",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.title("Distribution of Original versus Sampled Data",
          fontname = "sans", fontweight = "bold", fontsize = 15)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox = False, shadow = False, fontsize = 15)
dayofweek2 = ["0","Mon","Tue","Wed","Thurs","Fri","Sat","Sun"]
ax.set_xticklabels(dayofweek2, minor=False, rotation=0)

plt.show()

Plot the distribution of original versus sampled data for arrival delay.

In [ ]:
counts7 = ontime4b.groupby(["biarrdelay","char"]).count()
counts7 = counts7.iloc[:,0:1]

totals7 = counts7.sum(level=0)

counts7 = counts7.unstack(level=1)
counts7.columns = counts7.columns.droplevel(level=0)
counts7


fig, ax = plt.subplots(figsize = (12,9))
biarrdelay = ontime4b.biarrdelay.unique()
plt.bar(biarrdelay, counts7["sample"], bottom=None, color= "#1F78B4", label= "sample")
plt.bar(biarrdelay, counts7["original"], bottom = counts7["sample"], color="#A6CEE3", label= "original")
plt.legend()
plt.xlabel("Arrival Delay",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.ylabel("Count",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.title("Distribution of Original versus Sampled Data",
          fontname = "sans", fontweight = "bold", fontsize = 15)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox = False, shadow = False, fontsize = 15)
plt.locator_params(nbins=2)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.show()

### d. Pre-processing for predictive modelling

Convert categorical variables to dummy variables. Multicollinearity issues could be avoided by removing the last dummy of every variables.

In [ ]:
ontime5 = ontime4[["biarrdelay","crsdeptimeintervals","crsarrtimeintervals", "carrier", "destairport", "originairport",
            "month", "dayofweek"]]

In [ ]:
ontime5["dayofweek"] = pd.Categorical(ontime5.dayofweek)
ontime5["month"] = pd.Categorical(ontime5.dayofweek)
ontime5 = pd.get_dummies(ontime5, drop_first = True)

Select features

In [ ]:
X = ontime5.iloc[:, 1:]

Select target variable

In [ ]:
y = ontime5.iloc[:, 0]
y = y.astype("category")

Select 75% of the sampled data randomly as the training set, the rest of 25% of the data is used as test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state= 1)

# e. Classification models

### Logistic Regression

In [ ]:
lr = LogisticRegression(max_iter = 10000, penalty='none')
lr.fit(X_train, y_train);

### Penalised Logistic Regression

In [ ]:
plr = LogisticRegression(penalty='l1', max_iter = 10000, tol=0.01, solver='saga')
plr.fit(X_train, y_train);

### Gradient Boosting

In [ ]:
gdb = GradientBoostingClassifier(random_state = 2)
gdb.fit(X_train, y_train);

### Classfication Tree

In [ ]:
ct = DecisionTreeClassifier(random_state=0)
ct.fit(X_train, y_train);

### Random Forests

In [ ]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train);

Use grid search with cross validations to identify optimal hyperparamter of the model for most accurate prediction

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [50, 150, 300, 600]
}


grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

### ROC Curve

In [ ]:
ax, fig = plt.subplots(figsize=(8,8))
ax = plt.gca()

plot_roc_curve(lr, X_test, y_test, ax=ax, name='Logistic Regression')
plot_roc_curve(plr, X_test, y_test, ax=ax, name='Penalised logistic regression')
plot_roc_curve(gdb, X_test, y_test, ax=ax, name='Gradient Boosting')
plot_roc_curve(ct, X_test, y_test, ax=ax, name='Classification trees')
plot_roc_curve(rf, X_test, y_test, ax=ax, name='Random forests')
plt.plot([0, 1], [0, 1], color='black', lw=1, linestyle='--')
plt.legend(fontsize = 10)
plt.xlabel("False Positive Rate",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.ylabel("True Positive Rate",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.title("Comparison of Performance of Classification Models by ROC Curve",
          fontname = "sans", fontweight = "bold", fontsize = 15)
plt.xticks(fontname = "sans", fontweight = "light", fontsize = 15)
plt.yticks(fontname = "sans", fontweight = "light", fontsize = 15)

plt.show()